
# Introduction -- Capstone Project


The City of New York, is the most populous city in the United States. It is diverse and is the financial
capital of USA. It is multicultural. It provides lot of business oppourtunities and business friendly
environment. It has attracted many different players into the market. It is a global hub of business and
commerce. The city is a major center for banking and finance, retailing, world trade, transportation,
tourism, real estate, new media, traditional media, advertising, legal services, accountancy, insurance,
theater, fashion, and the arts in the United States.
This also means that the market is highly competitive. As it is highly developed city so cost of doing
business is also one of the highest. Thus, any new business venture or expansion needs to be analysed
carefully. The insights derived from analysis will give good understanding of the business
environment and the Return on Investment will be reasonable. 

## Business problem


The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history. 

Indian food have become so popular in the United States now it seems that there is  one on every corner, not only in major cities but also in smaller cities. Starting a sushi restaurant can be a great business opportunity, but you need to distinguish yourself from others to enjoy long-term success.

### Find a suitable location. 

If you plan a real restaurant that can demand higher prices for authentic indian food, focus on neighborhoods and outlets that already attract a sophisticated client. If you plan a cheap buffet restaurant, points to the masses looking for affordable high-traffic locations with large shopping centers and other local points of interest. So it is evident that to survive in such competitive market it is very important to startegically plan.
Various factors need to be studied inorder to decide on the Location 

### Target Audience: 

My Client wants to open his business in Manhattan area, so I only focus on that borough during my analysis. The objective is to locate and recommend to the management which neighborhood of Newyork city will be best choice to start a restaurant. The Management also expects to understand the rationale of the recommendations made.

This would interest anyone who wants to start a new restaurant in Newyork city.


### Data Source

To find the best location for our sushi bar, we will use the following sources of information:
<ul>
    
<li>Newyork has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572
</li>
<br>

<br>

<li>
From Foursquare 
    Venues Categories - <a> https://developer.foursquare.com/docs/resources/categories</a>

INDIAN category Id - 4bf58dd8d48988d10f941735

</li>
</ul>

## Import Libraries

In this section we import the libraries that will be required to process the data.

In [28]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Libraries imported.


## Download and Explore Dataset

In [29]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [30]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [31]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In [32]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [34]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [35]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues

In [36]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [37]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [39]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d10f941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_ind = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10f941735')
newyork_venues_ind.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Riverdale Indian Cuisine,40.880886,-73.908800,Indian Restaurant
1,Marble Hill,40.876551,-73.910660,Tazmohol Indian Restaurant,40.879331,-73.903192,Indian Restaurant
2,Marble Hill,40.876551,-73.910660,Cumin Indian Cuisine,40.886459,-73.909816,Indian Restaurant
3,Chinatown,40.715618,-73.994279,Kabab Bites,40.720094,-73.995819,Indian Restaurant
4,Chinatown,40.715618,-73.994279,Hampton Chutney Co.,40.720220,-73.999347,Dosa Place


In [40]:
newyork_venues_ind.shape

(1090, 7)

In [41]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [42]:
map_newyork_ind = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_ind, 'red', map_newyork_ind)

map_newyork_ind

In [43]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [45]:
manhattan_grouped = newyork_venues_ind.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_ind['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,25,25,25,25,25,25
Carnegie Hill,13,13,13,13,13,13
Central Harlem,10,10,10,10,10,10
Chelsea,17,17,17,17,17,17
Chinatown,25,25,25,25,25,25
Civic Center,30,30,30,30,30,30
Clinton,45,45,45,45,45,45
East Harlem,8,8,8,8,8,8
East Village,47,47,47,47,47,47


## 3. Analyze Each Neighborhood

In [46]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_ind[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_ind['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Himalayan Restaurant,Hookah Bar,Indian Chinese Restaurant,Indian Restaurant,Music Store,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Chinatown,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [47]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Himalayan Restaurant,Hookah Bar,Indian Chinese Restaurant,Indian Restaurant,Music Store,North Indian Restaurant,Pakistani Restaurant,South Indian Restaurant,Tapas Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.840000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.923077,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.058824,0.764706,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
4,Chinatown,0.000000,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.880000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.133333,0.000000,0.000000,0.000000,0.822222,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.021277,0.042553,0.000000,0.042553,0.000000,0.000000,0.000000,0.000000,0.851064,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277
9,Financial District,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.185185,0.000000,0.000000,0.000000,0.740741,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000


In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Indian Restaurant,Food Truck,Asian Restaurant,Tapas Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Vegetarian / Vegan Restaurant
1,Carnegie Hill,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
2,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
3,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
4,Chinatown,Indian Restaurant,North Indian Restaurant,Deli / Bodega,Dosa Place,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Diner,Hookah Bar


##Cluster Neighborhood

In [50]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 1, 2, 0, 2, 2, 1, 0, 2], dtype=int32)

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
1,Manhattan,Chinatown,40.715618,-73.994279,0.0,Indian Restaurant,North Indian Restaurant,Deli / Bodega,Dosa Place,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Diner,Hookah Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)

if cluster == 'nan':
    cluster =1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Indian Restaurant,North Indian Restaurant,Deli / Bodega,Dosa Place,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Diner,Hookah Bar
16,Murray Hill,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,North Indian Restaurant,Diner,Chaat Place,Deli / Bodega,Dosa Place
19,East Village,Indian Restaurant,Deli / Bodega,Dosa Place,Vegetarian / Vegan Restaurant,Chaat Place,North Indian Restaurant,Food Truck,Burrito Place,Diner,Hookah Bar
22,Little Italy,Indian Restaurant,Dosa Place,Asian Restaurant,North Indian Restaurant,Deli / Bodega,Food Truck,Burrito Place,Chaat Place,Diner,Vegetarian / Vegan Restaurant
23,Soho,Indian Restaurant,Food Truck,Dosa Place,Asian Restaurant,North Indian Restaurant,Deli / Bodega,Burrito Place,Chaat Place,Diner,Vegetarian / Vegan Restaurant
27,Gramercy,Indian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Hookah Bar,North Indian Restaurant,Tapas Restaurant
31,Noho,Indian Restaurant,Dosa Place,Vegetarian / Vegan Restaurant,Food Truck,Chaat Place,North Indian Restaurant,Deli / Bodega,Burrito Place,Diner,Hookah Bar
33,Midtown South,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Pakistani Restaurant,Burrito Place,Diner,Chaat Place,Deli / Bodega,Dosa Place
38,Flatiron,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Deli / Bodega,Diner,Hookah Bar,North Indian Restaurant,Tapas Restaurant


In [79]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
2,Washington Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
4,Hamilton Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
5,Manhattanville,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
6,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
7,East Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
25,Manhattan Valley,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar
26,Morningside Heights,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Hookah Bar


In [80]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Dosa Place,Hookah Bar
15,Midtown,Indian Restaurant,Food Truck,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Dosa Place,Burrito Place,Chaat Place
17,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
28,Battery Park City,Indian Restaurant,Food Truck,Asian Restaurant,Tapas Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Vegetarian / Vegan Restaurant
29,Financial District,Indian Restaurant,Food Truck,Asian Restaurant,Tapas Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Vegetarian / Vegan Restaurant
32,Civic Center,Indian Restaurant,Food Truck,Asian Restaurant,Dosa Place,Tapas Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Vegetarian / Vegan Restaurant
39,Hudson Yards,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Dosa Place,Hookah Bar


In [81]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Indian Restaurant,Vegetarian / Vegan Restaurant,Chaat Place,North Indian Restaurant,Deli / Bodega,Dosa Place,Food Truck,Burrito Place,Diner,Hookah Bar


In [82]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
9,Yorkville,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
10,Lenox Hill,Indian Restaurant,Asian Restaurant,North Indian Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Vegetarian / Vegan Restaurant
11,Roosevelt Island,Indian Restaurant,Asian Restaurant,Himalayan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Food Truck,Vegetarian / Vegan Restaurant
12,Upper West Side,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
13,Lincoln Square,Indian Restaurant,Food Truck,South Indian Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar
18,Greenwich Village,Indian Restaurant,Dosa Place,Food Truck,North Indian Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Hookah Bar
20,Lower East Side,Indian Restaurant,Vegetarian / Vegan Restaurant,Chaat Place,North Indian Restaurant,Deli / Bodega,Food Truck,Burrito Place,Diner,Dosa Place,Hookah Bar
21,Tribeca,Indian Restaurant,Asian Restaurant,Food Truck,Dosa Place,Tapas Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Vegetarian / Vegan Restaurant
24,West Village,Indian Restaurant,Food Truck,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Chaat Place,Deli / Bodega,Diner,Dosa Place,Hookah Bar


## Methodology 

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

In this project, I will use the basic methodology as taught in Week 3 lab.
I have done convert addresses into their equivalent latitude and longitude values. 
Then we will use the Foursquare API to explore neighborhoods in Manhattan, New York. 
After that, explore function to get indian restaurant categories in each neighborhood.
Then use this feature to group the neighborhoods into clusters K-means clustering algorithm will be use to complete this task. 
And also, the Folium library to visualize the neighborhoods in Manhattan and its emerging cluster

## Results 

K-mean Cluster Using K-mean to clustering data area with less number of sushi bars

Based on dataframe analysis above Cluster 3 Stuyvesant Town is best to open Indian restaurant

## Discussion 

Below are discussion points
There is high competition in other areas so risky tto open restaurant
It can be done more detailed analysis by adding other factors such as transportation, demographics of inhabitants.   
FourSquare proved to be a good source of data 

## Conclusion 

ALthough the prokect recommended the location but still there is room for improvement but still the goal to identify a location was made possible 